In [173]:
# Import some dependencies
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import pandas as pd

# GET target for the live dataset
url = 'https://serene-forest-99801.herokuapp.com/api/allData'

In [174]:
# Set up vectorizers to transform data
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer()
tfidf = TfidfTransformer()

In [175]:
# Import the live dataset
dataset = pd.read_json(url)

In [176]:
# An attempt to massage the dataset to produce better results
# * Needed to bucket categories so each had 10+ features

# Define variable sets for bucketing
man = ['Cis_Man']
woman = ['Cis_Woman']

# Regex for everything not in a defined bucket
regex = '^((?!(^Man$|^Woman$|^Other$)).).+$'

# Removes 'unspecified' since they are unknowable features
# and buckets all others into 'Man', 'Woman', and 'Other'
# for model evaluation
dataset = dataset[dataset.gender != 'unspecified']
dataset['gender'] = dataset['gender'].replace(man, 'Man')
dataset['gender'] = dataset['gender'].replace(woman, 'Woman')
dataset['gender'] = dataset['gender'].replace(
    to_replace=regex,
    value='Other',
    regex=True
)

In [177]:
# Vectorize the data as needed and split into `X` and `y` varibles for training / testing
X = tfidf.fit_transform(vectorizer.fit_transform(dataset.writing))

y = dataset.gender

In [181]:
# Test Bernoulli Naive Bayes with K-Fold (10-fold) cross-validation
from sklearn.naive_bayes import MultinomialNB
multiNB = MultinomialNB()
print(cross_val_score(multiNB, X, y, cv=10, scoring='accuracy').mean())

0.513888888889


In [180]:
# Test Multinomial Naive Bayes with K-Fold (10-fold) cross-validation
from sklearn.naive_bayes import BernoulliNB
bernoulliNB = BernoulliNB()
print(cross_val_score(bernoulliNB, X, y, cv=10, scoring='accuracy').mean())

0.494444444444


In [179]:
# Test Logistic Regression with K-Fold (10-fold) cross-validation
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
print(cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean())

0.477777777778
